# Features

In [1]:
import pandas as pd

Trabajo en el siguiente rango de dias:

In [2]:
firstDay = pd.Timestamp(2019, 4, 24)
lastDay = pd.Timestamp(2019, 4, 27)

Los datos son los siguientes

In [3]:
location = 'data/'
installsFile = location + 'installs.csv'

Cargo los datos de los primeros 3 dias

## Installs

In [4]:
dtype = {
    'created': 'str',
    'application_id': 'category',
    'ref_type': 'category',
    'ref_hash': 'int64',
    'click_hash': 'str',
    'attributed': 'bool',
    'implicit': 'bool',
    'device_brand': 'float64',
    'device_model': 'float64',
    'session_user_agent': 'str',
    'user_agent': 'str',
    'event_uuid': 'str',
    'kind': 'str',
    'wifi': 'str',
    'trans_id': 'str',
    'ip_address': 'int64',
    'device_language': 'category',
}

In [5]:
installs = pd.read_csv(installsFile, usecols=dtype.keys(), dtype=dtype, parse_dates=['created'])

In [6]:
installsIsInDateInterval = (installs['created'] >= firstDay) & (installs['created'] < lastDay)
installs = installs[installsIsInDateInterval]

In [7]:
installs = installs.rename(columns={'ref_hash': 'device_id'})

In [8]:
installs['seconds'] = (installs['created'] - firstDay).dt.total_seconds()
installs['day'] = installs['created'].dt.day - firstDay.day + 1

In [9]:
installs.head()

,created,application_id,ref_type,device_id,click_hash,attributed,implicit,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language,seconds,day
0,2019-04-24 06:23:29.495,1,1494519392962156891,4716708407362582887,NaN,False,True,NaN,3.739127e+17,adjust.com,NaN,79837499-2f2a-4605-a663-e322f759424f,app_open,NaN,NaN,4243443387795468703,3.3013777759776993e+18,23009.495,1
1,2019-04-24 02:06:01.032,1,1494519392962156891,7143568733100935872,NaN,False,False,NaN,7.805539e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,4724288679627032761,3.3013777759776993e+18,7561.032,1
29,2019-04-26 12:17:29.831,1,1494519392962156891,8326131692284822715,NaN,False,True,NaN,4.780545e+18,adjust.com,NaN,c7c0f1c7-b5bd-4887-8c15-22faa9eed45a,app_open,NaN,NaN,7914241434760640009,3.3013777759776993e+18,217049.831,3
33,2019-04-24 16:08:49.392,3,1891515180541284343,7982672190758515108,NaN,False,False,NaN,7.047732e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,2331161058149107059,3.3013777759776993e+18,58129.392,1
34,2019-04-24 01:43:29.697,3,1891515180541284343,2374271566729163309,NaN,False,False,NaN,3.017401e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,4368472372578406700,3.3013777759776993e+18,6209.697,1


In [10]:
installs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 160033 entries, 0 to 481509
Data columns (total 19 columns):
created               160033 non-null datetime64[ns]
application_id        160033 non-null category
ref_type              160033 non-null category
device_id             160033 non-null int64
click_hash            523 non-null object
attributed            160033 non-null bool
implicit              160033 non-null bool
device_brand          88505 non-null float64
device_model          150272 non-null float64
session_user_agent    153205 non-null object
user_agent            108706 non-null object
event_uuid            33265 non-null object
kind                  33265 non-null object
wifi                  95821 non-null object
trans_id              3219 non-null object
ip_address            160033 non-null int64
device_language       149452 non-null category
seconds               160033 non-null float64
day                   160033 non-null int64
dtypes: bool(2), category(3), dat

In [11]:
features = pd.DataFrame(installs['device_id'].unique(), columns=['device_id'])

In [12]:
features.head()

,device_id
0,4716708407362582887
1,7143568733100935872
2,8326131692284822715
3,7982672190758515108
4,2374271566729163309


In [13]:
installsCount = installs[['device_id']].groupby(by=['device_id']).size()
installsCount = installsCount.reset_index().rename(columns={0: 'installsCount'})
features = features.merge(installsCount, how='left')

In [14]:
installsMostFreqDay = installs[['device_id', 'day']].groupby(by=['device_id']).agg(lambda x: x.value_counts().index[0])
installsMostFreqDay = installsMostFreqDay.reset_index().rename(columns={'day': 'installsMostFreqDay'})
features = features.merge(installsMostFreqDay, how='left')

In [15]:
installsMeanInterval = installs[['device_id', 'seconds']].sort_values(by='seconds').groupby(by='device_id').agg(lambda x: (x['seconds'] - x['seconds'].shift(1).fillna(0)).mean())
installsMeanInterval = installsMeanInterval.reset_index().rename(columns={'seconds': 'installsMeanInterval'})
features = features.merge(installsMeanInterval, how='left')

In [16]:
features['installsAproxFreq'] = features['installsMeanInterval']**-1

In [17]:
installsStdDevInterval = installs[['device_id', 'seconds']].sort_values(by='seconds').groupby(by='device_id').agg(lambda x: (x['seconds'] - x['seconds'].shift(1).fillna(0)).std())
installsStdDevInterval = installsStdDevInterval.reset_index().rename(columns={'seconds': 'installsStdDevInterval'})
features = features.merge(installsStdDevInterval, how='left')

In [18]:
installsLast = installs[['device_id', 'seconds']].groupby(by='device_id').max()
installsLast = installsLast.reset_index().rename(columns={'seconds': 'installsLast'})
features = features.merge(installsLast, how='left')

In [19]:
features.head()

,device_id,installsCount,installsMostFreqDay,installsMeanInterval,installsAproxFreq,installsStdDevInterval,installsLast
0,4716708407362582887,1,1,23009.495,0.000043,NaN,23009.495
1,7143568733100935872,1,1,7561.032,0.000132,NaN,7561.032
2,8326131692284822715,1,3,217049.831,0.000005,NaN,217049.831
3,7982672190758515108,1,1,58129.392,0.000017,NaN,58129.392
4,2374271566729163309,1,1,6209.697,0.000161,NaN,6209.697


In [20]:
features.to_csv('data/installsFeatures7.csv', index=False)